In [16]:
import cv2
import numpy as np
import mtcnn
from architecture import *
from train_v2 import normalize, l2_normalizer
from scipy.spatial.distance import cosine
from tensorflow.keras.models import load_model
import pickle
import mediapipe as mp
import random
import math
import time

import os
from pathlib import Path
from facetools import FaceDetection
from facetools import IdentityVerification
from facetools import LivenessDetection
from facetools.utils import visualize_results

In [17]:
root = Path(os.path.abspath(__file__)).parent.absolute()
data_folder = root / "data"




NameError: name '__file__' is not defined

In [ ]:
resNet_checkpoint_path = data_folder / "checkpoints" / "InceptionResnetV1_vggface2.onnx"
facebank_path = data_folder / "reynolds.csv"

deepPix_checkpoint_path = data_folder / "checkpoints" / "OULU_Protocol_2_model_0_0.onnx"

faceDetector = FaceDetection(max_num_faces=1)
identityChecker = IdentityVerification(
    checkpoint_path=resNet_checkpoint_path.as_posix(),
    facebank_path=facebank_path.as_posix(),
)
livenessDetector = LivenessDetection(checkpoint_path=deepPix_checkpoint_path.as_posix())

NameError: name 'data_folder' is not defined

In [ ]:
# Initialize face detection and recognition configurations
confidence_t = 0.99
recognition_t = 0.5
required_size = (160, 160)

# Initialize MediaPipe solutions
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic

# Initialize MediaPipe Holistic Model for hand tracking
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.7)

mp_hands = mp.solutions.hands
hands= mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

recognized_name = 'unknown'

In [ ]:
# Function to extract and encode faces
def get_face(img, box):
    x1, y1, width, height = box
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)

def get_encode(face_encoder, face, size):
    face = normalize(face)
    face = cv2.resize(face, size)
    encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
    return encode


def load_pickle(path):
    with open(path, 'rb') as f:
        encoding_dict = pickle.load(f)
    return encoding_dict

def detect(img ,detector,encoder,encoding_dict):
    global recognized_name  # Declare the global variable
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(img_rgb)
    for res in results:
        if res['confidence'] < confidence_t:
            continue
        face, pt_1, pt_2 = get_face(img_rgb, res['box'])
        encode = get_encode(encoder, face, required_size)
        encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
        name = 'unknown'

        distance = float("inf")
        for db_name, db_encode in encoding_dict.items():
            dist = cosine(db_encode, encode)
            if dist < recognition_t and dist < distance:
                name = db_name
                distance = 1 - dist

        if name == 'unknown':
            cv2.rectangle(img, pt_1, pt_2, (0, 0, 255), 2)
            cv2.putText(img, name, pt_1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
        else:
            recognized_name = name  # Update the recognized name
            cv2.rectangle(img, pt_1, pt_2, (0, 255, 0), 2)
            cv2.putText(img, name + f'__{distance:.2f}', (pt_1[0], pt_1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 200, 200), 2)
    return img


In [ ]:
# Function to get fingertip position from hand landmarks
def get_fingertip_position(hand_landmarks):
    # Assuming index fingertip is used for pointing
    fingertip = hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP]
    return int(fingertip.x * frame_width), int(fingertip.y * frame_height)


# Function to calculate distance between two landmarks
def calculate_distance(landmark1, landmark2):
    return math.sqrt((landmark1.x - landmark2.x) ** 2 + (landmark1.y - landmark2.y) ** 2)

In [ ]:
# Function to extract and encode faces
def get_face(img, box):
    x1, y1, width, height = box
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)

def get_encode(face_encoder, face, size):
    face = normalize(face)
    face = cv2.resize(face, size)
    encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
    return encode


def load_pickle(path):
    with open(path, 'rb') as f:
        encoding_dict = pickle.load(f)
    return encoding_dict

def detect(img ,detector,encoder,encoding_dict):
    global recognized_name  # Declare the global variable
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(img_rgb)
    for res in results:
        if res['confidence'] < confidence_t:
            continue
        face, pt_1, pt_2 = get_face(img_rgb, res['box'])
        encode = get_encode(encoder, face, required_size)
        encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
        name = 'unknown'

        distance = float("inf")
        for db_name, db_encode in encoding_dict.items():
            dist = cosine(db_encode, encode)
            if dist < recognition_t and dist < distance:
                name = db_name
                distance = 1 - dist

        if name == 'unknown':
            cv2.rectangle(img, pt_1, pt_2, (0, 0, 255), 2)
            cv2.putText(img, name, pt_1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
        else:
            recognized_name = name  # Update the recognized name
            cv2.rectangle(img, pt_1, pt_2, (0, 255, 0), 2)
            cv2.putText(img, name + f'__{distance:.2f}', (pt_1[0], pt_1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 200, 200), 2)
    return img



# Function to get fingertip position from hand landmarks
def get_fingertip_position(hand_landmarks):
    # Assuming index fingertip is used for pointing
    fingertip = hand_landmarks.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP]
    return int(fingertip.x * frame_width), int(fingertip.y * frame_height)


# Function to calculate distance between two landmarks
def calculate_distance(landmark1, landmark2):
    return math.sqrt((landmark1.x - landmark2.x) ** 2 + (landmark1.y - landmark2.y) ** 2)



# Function to check if hand is showing an 'OK' sign
def is_ok_sign(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    index_pip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP]
    
    # Check if the thumb tip is near the index finger PIP joint (making a circle)
    # and if the index finger tip is near the thumb tip.
    thumb_index_distance = np.sqrt((thumb_tip.x - index_tip.x) ** 2 + 
                                   (thumb_tip.y - index_tip.y) ** 2 +
                                   (thumb_tip.z - index_tip.z) ** 2)
    index_pip_distance = np.sqrt((index_tip.x - index_pip.x) ** 2 + 
                                 (index_tip.y - index_pip.y) ** 2 +
                                 (index_tip.z - index_pip.z) ** 2)
    
    if thumb_index_distance < 0.05 and thumb_index_distance < index_pip_distance:  # Threshold might need adjustment
        return True
    return False

# Function to check if hand is showing a rock sign
def is_rock_sign(hand_landmarks):
    # Index and little fingers should be extended, middle and ring fingers not extended
    return (hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y < 
            hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y and
            hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y < 
            hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP].y and
            hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y > 
            hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y and
            hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y > 
            hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP].y)

# Function to check if hand is showing a thumbs-up sign
def is_thumbs_up(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    
    # Thumb should be extended and above the MCP joint of the index finger
    if thumb_tip.y < thumb_mcp.y and thumb_tip.y < index_mcp.y:
        return True
    return False

# Function to check if hand is showing a thumbs-down sign
def is_thumbs_down(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
    index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    
    # Thumb should be extended and below the MCP joint of the index finger
    if thumb_tip.y > thumb_mcp.y and thumb_tip.y > index_mcp.y:
        return True
    return False

# Function to check if hand is showing a peace sign
def is_peace_sign(hand_landmarks):
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
    
    # Index and middle fingers should be extended, above the MCP of the ring finger
    if index_tip.y < ring_mcp.y and middle_tip.y < ring_mcp.y:
        return True
    return False

In [ ]:
# Function to randomly choose between a hand gesture and mouth state
def choose_random_prompt():
    prompts = ['Thumbs Up', 'Thumbs Down', 'Peace Sign', 'Rock Sign', 'OK Sign', 'Mouth Open', 'Mouth Closed']
    return random.choice(prompts)

displayed_prompt = choose_random_prompt()

In [ ]:
if __name__ == "__main__":
    face_encoder = InceptionResNetV2()
    path_m = "facenet_keras_weights.h5"
    face_encoder.load_weights(path_m)
    encodings_path = 'encodings/encodings.pkl'
    face_detector = mtcnn.MTCNN()
    encoding_dict = load_pickle(encodings_path)

    cap = cv2.VideoCapture(0)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


    start_time = time.time()  # Record the start time

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("CAM NOT OPENED")
            break

        display = 'No match'
        liveness_result = 'Not Checked'

        

        canvas = frame.copy()
        faces, boxes = faceDetector(frame)
        for face_arr, box in zip(faces, boxes):
            min_sim_score, mean_sim_score = identityChecker(face_arr)
            liveness_score = livenessDetector(face_arr)
            # Check if liveness score is greater than 0.03
            liveness_result = 'Real' if liveness_score > 0.03 else 'Fake'
            canvas = visualize_results(canvas, box, liveness_score, liveness_result)
       

        # Face detection and recognition
        frame = detect(canvas, face_detector, face_encoder, encoding_dict)

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process with both Hands and Holistic models
        hands_results = hands.process(frame_rgb)
        holistic_results = holistic.process(frame_rgb)

        # Convert back to BGR for rendering
        frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

        # Display the chosen prompt on screen
        cv2.putText(frame_bgr, f'Match this: {displayed_prompt}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        match_found = False

        # Hand gesture recognition
        if hands_results.multi_hand_landmarks and not match_found:
            for hand_landmarks in hands_results.multi_hand_landmarks:
                if is_thumbs_up(hand_landmarks):
                    gesture = 'Thumbs Up'
                elif is_thumbs_down(hand_landmarks):
                    gesture = 'Thumbs Down'
                elif is_peace_sign(hand_landmarks):
                    gesture = 'Peace Sign'
                elif is_rock_sign(hand_landmarks):
                    gesture = 'Rock Sign'
                elif is_ok_sign(hand_landmarks):
                    gesture = 'OK Sign'
                else:
                    gesture = 'Unknown Gesture'

                

                # Check if user's gesture matches the displayed gesture
                if gesture == displayed_prompt:
                    cv2.putText(frame_bgr, 'Match!', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    display='Match!'
                    match_found = True
                    break

        # Mouth state recognition
        if holistic_results.face_landmarks and not match_found:
            upper_lip = holistic_results.face_landmarks.landmark[13]  # Top of upper lip
            lower_lip = holistic_results.face_landmarks.landmark[14]  # Bottom of lower lip
            mouth_distance = calculate_distance(upper_lip, lower_lip)
            
            mouth_state = 'Mouth Closed'
            if mouth_distance > 0.03:  # Threshold for open mouth
                mouth_state = 'Mouth Open'

            if mouth_state == displayed_prompt:
                cv2.putText(frame_bgr, 'Match!', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                display='Match!'
            else:
                cv2.putText(frame_bgr, 'No match', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Check if 15 seconds have elapsed
        elapsed_time = time.time() - start_time
        if elapsed_time >= 30:
            print("TIMEOUT!!!!. Retry....")

            # Load and display success image
            error_image_path = 'timeout.jpg'  # Path to your success image
            error_image = cv2.imread(error_image_path)
            if error_image is not None:
                cv2.imshow('Error', error_image)
                cv2.waitKey(5000)  # Display the image for 5000 milliseconds (5 seconds)
                cv2.destroyWindow('Error')
            break


        # Check conditions and display/print "Welcome" message
        if recognized_name != 'unknown' and display == 'Match!' and liveness_result == 'Real':
            welcome_message = "Welcome"
            print(welcome_message)
            

            # Load and display success image
            success_image_path = 'image.jpg'  # Path to your success image
            success_image = cv2.imread(success_image_path)
            if success_image is not None:
                cv2.imshow('Success', success_image)
                cv2.waitKey(7000)  # Display the image for 5000 milliseconds (5 seconds)
                cv2.destroyWindow('Success')

            break



        cv2.imshow('camera', frame_bgr)
        
        

        # Press 'n' to display a new random prompt
        if cv2.waitKey(10) & 0xFF == ord('n'):
            displayed_prompt = choose_random_prompt()
            match_found = False
        
        # Exit the loop on pressing 'q'
        elif cv2.waitKey(10) & 0xFF == ord('q'):
            break
        

    cap.release()
    cv2.destroyAllWindows()

NameError: name 'faceDetector' is not defined